In [93]:
import glob
import random
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import pandas as pd
import joblib
from numpy.lib.stride_tricks import sliding_window_view

In [120]:
max_shifts_per_day = 2
shift_day_of_week = []
days = ['Monday','Tuesday','Wednesday','Thursday','Friday']

shift_id = [1,2,3,4]
i=0
while i < len(shift_id):
#for i in range(len(shift_id)):
    choice = random.choice(days)
    print(choice)
    if shift_day_of_week.count(choice) < max_shifts_per_day + 1:
        shift_day_of_week.append(choice)
        i+1
    print(i)


shift_day_of_week

KeyboardInterrupt: 

In [116]:
test_data  = pd.DataFrame(columns=['problem','reward', 'model', 'seed'])
test_data.to_csv('test_data/test_data.csv',index=False)


In [89]:
class SchedulingEnv(Env):
    """A personnel scheduling environment for OpenAI gym"""

    def __init__(self, pool, schedule, reward_type):
        sfEncodings = joblib.load('shiftFeatureEncoding.joblib')
        shifts = pd.get_dummies(schedule[['shift_id']],drop_first=True)
        sfEncoded =  sfEncodings.transform(schedule[['shift_day_of_week','shift_type']])
        shift_features = pd.DataFrame(sfEncoded, columns=sfEncodings.get_feature_names_out())
        schedule = pd.merge(shifts, shift_features, left_index=True, right_index=True)

        self.shift_features = schedule.shape[1]
        # OR self.shift_features = shift_features.shape[1]

        for i in pd.get_dummies(pool).columns.to_list():
            schedule[i] = 0
        
        self.schedule = schedule
        self.state = self.schedule.to_numpy()

        # sf_index may not be needed here
        sf_start = len(schedule)-1
        sf_end = schedule.shape[1] - len(pool)
        self.sf_index = (sf_start,sf_end)

        self.count_workers = len(pool)
        self.count_shifts = len(schedule)
        self.shift_number = 0
        self.reward_type = reward_type
        self.reward_step = 0
        self.cummulative_reward = 0
        self.cummulative_violations = 0
        
        # action space: Employees we can assign to shifts
        self.action_space = Discrete(self.count_workers)
        # observation space: the latest state matrix
        self.observation_space = Box(low=0, high=1, shape=(self.state.shape[0], self.state.shape[1]),\
                                     dtype=np.float64)

In [86]:
i = ('scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_01.csv', 'scheduling_problems/test_set/shifts_easy_ratio_mixed/pool_01.csv')

schedule, pool = pd.read_csv(f'{i[0]}',dtype={'employee_id':'str'}), \
                pd.read_csv(f'{i[1]}',dtype={'shift_id':'str'})

In [106]:
schedule['shift_day_of_week'] = schedule['shift_day_of_week'].replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],[1, 2, 3, 4, 5])
schedule['shift_type'] = schedule['shift_type'].replace(['Morning', 'Evening'],[1, 2])
schedule['shift_id'] = schedule['shift_id'].astype('str')

env = SchedulingEnv(pool, schedule, reward_type='Step_Bonus')

In [107]:
env.state

array([[0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.]])

## test problem list

In [83]:
def testProbList(subset):
    test_set = sorted(glob.glob(f"scheduling_problems/test_set/{subset}/*.csv"))
    test_index = pd.read_csv(f"scheduling_problems/test_set_indexes/testproblemindex_{subset}.csv")
    problist = []

    for i in range(len(test_index)):
        s = str(int(test_index.iloc[i]['Schedule'])).zfill(2)
        p = str(int(test_index.iloc[i]['Pool'])).zfill(2)

        sp = f"scheduling_problems/test_set/{subset}/schedule_{s}.csv"
        pp = f"scheduling_problems/test_set/{subset}/pool_{p}.csv"

        problist.append((sp,pp))

    return problist


In [117]:
str(i[0].split('_')[-1].split('.')[0] + i[1].split('_')[-1].split('.')[0])

'0101'

In [84]:
for i in testProbList('shifts_easy_ratio_mixed'):
    print(i[0])

scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_01
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_02
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_03
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_04
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_05
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_06
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_07
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_08
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_09
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_10
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_11
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_12
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_13
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_14
scheduling_problems/test_set/shifts_easy_ratio_mixed/schedule_15
scheduling_problems/test_

In [72]:
test_index.iloc[0]#['Schedule']
s = str(int(test_index.iloc[0]['Schedule'])).zfill(2)
p = str(int(test_index.iloc[0]['Pool'])).zfill(2)

dir=''
sp = f"scheduling_problems/test_set/{dir}/schedule_{s}"
pp = f"scheduling_problems/test_set/{dir}/pool_{p}"

(sp,pp)


'scheduling_problems/test_set//schedule_01'

In [69]:
test_set[51]

'scheduling_problems/test_set/schedule_02.csv'

In [54]:
test_set = sorted(glob.glob("scheduling_problems/test_set/*.csv"))
test_index = pd.read_csv("scheduling_problems/testproblemindex.csv")
test_index

for i in range(len(test_index):
    test_index.iloc
    {str(id).zfill(2)}

,Schedule,shifts,Pool,employees,nodes,ratio
0,1,3,1,2,5,1.500000
1,2,3,2,2,5,1.500000
2,3,7,3,3,10,2.333333
3,4,3,4,2,5,1.500000
4,5,7,5,4,11,1.750000
5,6,5,6,4,9,1.250000
6,7,6,7,3,9,2.000000
7,8,8,8,3,11,2.666667
8,9,5,9,4,9,1.250000
9,10,8,10,4,12,2.000000


In [53]:
loadTestProblem(num_shifts=False)

('scheduling_problems/test_set/pool_01.csv',
 'scheduling_problems/test_set/schedule_01.csv')

In [50]:
def loadTestProblem(n=False, num_shifts=False):
    tstst = sorted(glob.glob("scheduling_problems/test_set/*.csv"))

    try:
        if num_shifts:
            shfts = 0
            while not shfts == num_shifts:
                n = random.randint(1, (len(tstst) / 2))
                p = tstst[n-1]
                s = tstst[n+(int(len(tstst) / 2)-1)]

                schedule = pd.read_csv(f'{tstst[n+(int(len(tstst) / 2)-1)]}',dtype={'shift_id':'str'})
                if len(schedule) == num_shifts:
                    shfts = num_shifts

        else:
            n = random.randint(1, (len(tstst) / 2))
            p = tstst[n-1]
            s = tstst[n+(int(len(tstst) / 2)-1)]

        return (p,s)

    except ValueError as verror:
        print(verror)
        

    except NameError as nerror:
        print(nerror)
        

    except:
        print(f"No problem with {num_shifts} shifts.")

In [23]:
def testProblemIndex():
    tstst = sorted(glob.glob("scheduling_problems/test_set/*.csv"))

    tpi  = pd.DataFrame(columns=['Schedule','shifts', 'Pool', 'employees','nodes','ratio'])

    for i in range(int(len(tstst)/2)):
        pool = tstst[i]
        schedule = tstst[i+(int(len(tstst) / 2))]

        pool, schedule = pd.read_csv(pool,dtype={'employee_id':'str'}), \
                    pd.read_csv(schedule,dtype={'shift_id':'str'})
        
        Schedule = tstst[i].split('/')[2].split('_')[1].split('.')[0]
        shifts = int(len(schedule))
        Pool = tstst[i+(int(len(tstst) / 2))].split('/')[2].split('_')[1].split('.')[0]
        employees = int(len(pool))
        nodes = shifts + employees 
        ratio = shifts / employees

        tpi.loc[0 if pd.isnull(tpi.index.max()) else tpi.index.max() + 1] = [Schedule] + [shifts] + [Pool] + [employees] + [nodes] + [ratio]

    sdf = df.groupby(['shifts']).agg({
    'shifts': 'count',
    'nodes': 'mean',
    'ratio': 'mean',
    'employees': 'mean'
        })

    sdf.columns =['count','avg_nodes','avg_ratio','avg_employees']


    return tpi, sdf


In [39]:
 df = testProblemIndex()
 df

,Schedule,shifts,Pool,employees,nodes,ratio
0,01,3,01,2,5,1.500000
1,02,3,02,2,5,1.500000
2,03,7,03,3,10,2.333333
3,04,3,04,2,5,1.500000
4,05,7,05,4,11,1.750000
5,06,5,06,4,9,1.250000
6,07,6,07,3,9,2.000000
7,08,8,08,3,11,2.666667
8,09,5,09,4,9,1.250000
9,10,8,10,4,12,2.000000
